In [2]:

%matplotlib inline

from pathlib import Path
from datetime import date
import json
from io import BytesIO
from zipfile import ZipFile, BadZipFile
from tqdm import tqdm
import requests
import numpy as np
import pandas as pd

from pprint import pprint

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import os
import multiprocessing

In [3]:
sns.set_style('whitegrid')
# store data in this directory since we won't use it in other chapters
data_path = Path(os.getcwd()) # perhaps set to external harddrive to accomodate large amount of data

In [ ]:
import time
# store data in this directory since we won't use it in other chapters
data_path = Path(os.getcwd()) # perhaps set to external harddrive to accomodate large amount of data
all_stocks = pd.read_csv("All_Stock.csv") 
all_symbol = all_stocks["Symbol"].unique()
all_symbol = [str(s) for s in all_symbol if (str(s).isalpha())]

path = data_path / 'company financials'
file_list = os.listdir(path)

year_limit = 2007
year_count = 15
errors=[]
for i in file_list:
    path = data_path /"fundamental data"/ 'company financials'/i
    if not path.exists():
        path.mkdir(parents=True)
    url_is = "https://fmpcloud.io/api/v3/income-statement/{}?limit={}&apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i, year_count)
    response_is = requests.get(url_is)
    company_data=pd.DataFrame()
    company_data.index.names = ['Year']
    for item in response_is.json():
        if int(item["calendarYear"]) >= year_limit:
            company_data.loc[int(item["calendarYear"]), "Date"] = item["date"]
            company_data.loc[int(item["calendarYear"]), "FillingDate"] = item["fillingDate"]
            company_data.loc[int(item["calendarYear"]), "NetIncome"] = item["netIncome"]
            company_data.loc[int(item["calendarYear"]), "Revenue"] = item["revenue"]
            company_data.loc[int(item["calendarYear"]), "OperatingIncome"] = item["operatingIncome"]
            company_data.loc[int(item["calendarYear"]), "EPSDiluted"] = item["epsdiluted"]
            company_data.loc[int(item["calendarYear"]), "GrossProfit"] = item["grossProfit"]
            company_data.loc[int(item["calendarYear"]), "GrossProfitRatio"] = item["grossProfitRatio"]
    if response_is.status_code != 200:
        errors.append(i)
    # time.sleep(0.01)

    url_bs = "https://fmpcloud.io/api/v3/balance-sheet-statement/{}?limit={}&apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i, year_count)
    response_bs = requests.get(url_bs)
    for item in response_bs.json():
        if int(item["calendarYear"]) >= year_limit:
            company_data.loc[int(item["calendarYear"]), "TotalAssets"] = item["totalAssets"]
            company_data.loc[int(item["calendarYear"]), "CommonStockEquity"] = item["totalStockholdersEquity"]
            company_data.loc[int(item["calendarYear"]), "CurrentAssets"] = item["totalCurrentAssets"]
            company_data.loc[int(item["calendarYear"]), "CurrentLiability"] = item["totalCurrentLiabilities"]
            company_data.loc[int(item["calendarYear"]), "TotalLiability"] = item["totalLiabilities"]
            company_data.loc[int(item["calendarYear"]), "Inventory"] = item["inventory"]
            company_data.loc[int(item["calendarYear"]), "RetainedEarnings"] = item["retainedEarnings"]
    if response_bs.status_code != 200:
        errors.append(i)
    # time.sleep(0.01)

    if len(company_data) == 0:
        errors.append(i + " no data")
        continue

    # url_ev = "https://fmpcloud.io/api/v3/enterprise-values/{}?limit={}&apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i, year_count)
    # response_ev = requests.get(url_ev)
    # for item in response_ev.json():
    #     year = int(item["date"][0:4])
    #     if year >= year_limit:
    #         company_data.loc[year, "MarketCap"] = item["marketCapitalization"]
    #         company_data.loc[year, "EnterpriseValue"] = item["enterpriseValue"]
    # if response_ev.status_code != 200:
    #     errors.append(i)
    # time.sleep(0.01)

    url_kr_ttm = "https://fmpcloud.io/api/v3/key-metrics-ttm/{}?limit=40&apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i)
    response_kr_ttm = requests.get(url_kr_ttm)
    if response_kr_ttm.json():
        for item in ["enterpriseValueTTM",'bookValuePerShareTTM', 'currentRatioTTM','debtToAssetsTTM','debtToEquityTTM', 'roicTTM','shareholdersEquityPerShareTTM','roeTTM','priceToSalesRatioTTM','peRatioTTM','pbRatioTTM','payoutRatioTTM','inventoryTurnoverTTM','enterpriseValueTTM','dividendYieldTTM']:
            company_data.loc["TTM", item[0:-3]] = response_kr_ttm.json()[0][item]
    else:
        errors.append(i)
    if response_kr_ttm.status_code != 200:
        errors.append(i)
    # time.sleep(0.01)

    url_kr = "https://fmpcloud.io/api/v3/key-metrics/{}?limit={}&apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i, year_count)
    response_kr = requests.get(url_kr)
    for item in response_kr.json():
        year = int(item["date"][0:4])
        if year >= year_limit:
            for key in ["enterpriseValueTTM",'bookValuePerShareTTM', 'currentRatioTTM','debtToAssetsTTM','debtToEquityTTM', 'roicTTM','shareholdersEquityPerShareTTM','roeTTM','priceToSalesRatioTTM','peRatioTTM','pbRatioTTM','payoutRatioTTM','inventoryTurnoverTTM','enterpriseValueTTM','dividendYieldTTM']:
                company_data.loc[year, key[0:-3]] = item[key[0:-3]]
    if response_kr.status_code != 200:
        errors.append(i)
    # time.sleep(0.01)

    url_fr = "https://fmpcloud.io/api/v3/ratios/{}?limit={}&apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i, year_count)
    response_fr = requests.get(url_fr)
    for item in response_fr.json():
        year = int(item["date"][0:4])
        if year >= year_limit:
            for key in ['assetTurnoverTTM',"quickRatioTTM", 'returnOnAssetsTTM','priceToFreeCashFlowsRatioTTM','grossProfitMarginTTM']:
                company_data.loc[year, key[0:-3]] = item[key[0:-3]]
    if response_fr.status_code != 200:
        errors.append(i)
    # time.sleep(0.01)

    url_fr_ttm = "https://fmpcloud.io/api/v3/ratios-ttm/{}?apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i)
    response_fr_ttm = requests.get(url_fr_ttm)
    if response_fr_ttm.json():
        for item in ['assetTurnoverTTM',"quickRatioTTM", 'returnOnAssetsTTM','priceToFreeCashFlowsRatioTTM','grossProfitMarginTTM']:
            company_data.loc["TTM", item[0:-3]] = response_fr_ttm.json()[0][item]
    else:
        errors.append(i)
    if response_fr_ttm.status_code != 200:
        errors.append(i)
    # time.sleep(0.01)

    url_fsg = "https://fmpcloud.io/api/v3/financial-growth/{}?limit={}&apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i, year_count)
    response_fsg = requests.get(url_fsg)
    for item in response_fsg.json():
        year = int(item["date"][0:4])
        if year >= year_limit:
            for key in ['epsdilutedGrowth',"fiveYNetIncomeGrowthPerShare", 'fiveYRevenueGrowthPerShare','fiveYShareholdersEquityGrowthPerShare','grossProfitGrowth', "netIncomeGrowth","operatingIncomeGrowth","threeYNetIncomeGrowthPerShare","threeYRevenueGrowthPerShare","threeYShareholdersEquityGrowthPerShare"]:
                company_data.loc[year, key] = item[key]
    if response_fsg.status_code != 200:
        errors.append(i)
    # time.sleep(0.01)

    url_cp = "https://fmpcloud.io/api/v3/profile/{}?apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i)
    response_cp = requests.get(url_cp)
    if response_cp.json():
        company_data["Industry"] = response_cp.json()[0]["industry"]
        company_data["Sector"] = response_cp.json()[0]["sector"]
        company_data["Price"] = response_cp.json()[0]["price"]
        company_data["MarketCap"] = response_cp.json()[0]["mktCap"]
        company_data["AvgVolume"] = response_cp.json()[0]["volAvg"]
    else:
        errors.append(i)
    
    company_data["Symbol"] = i
    if response_cp.status_code != 200:
        errors.append(i)
    # time.sleep(0.01)

    company_data = company_data.rename(columns={"enterpriseValue":"EnterpriseValue"})
    file = "{}_fundamentals_Year.csv".format(i)
    company_data.to_csv(data_path /"fundamental data"/ 'company financials'/i/file)
    print(i, "Done")

In [ ]:
all_stocks = pd.read_csv("All_Stock.csv") 
all_symbol = all_stocks["Symbol"].unique()
bad_symbol = [i.split()[0] for i in errors]
all_symbol = [str(s) for s in all_symbol if (str(s).isalpha())  and (str(s) not in bad_symbol)]

year_limit = 2007
quarter_count = 70
errors_q=[]
for i in all_symbol:
    path = data_path /"fundamental data"/ 'company financials'/i
    if not path.exists():
        path.mkdir(parents=True)
    url_is = "https://fmpcloud.io/api/v3/income-statement/{}?period=quarter&limit={}&apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i, quarter_count)
    response_is = requests.get(url_is)
    company_data=pd.DataFrame()
    company_data.index.names = ['Quarters']
    for item in response_is.json():
        if int(item["calendarYear"]) >= year_limit:
            company_data.loc[item["calendarYear"]+item["period"], "Year"] = item["calendarYear"]
            company_data.loc[item["calendarYear"]+item["period"], "Quarter"] = item["period"]
            company_data.loc[item["calendarYear"]+item["period"], "FillingDate"] = item["fillingDate"]
            company_data.loc[item["calendarYear"]+item["period"], "Date"] = item["date"]
            company_data.loc[item["calendarYear"]+item["period"], "NetIncome"] = item["netIncome"]
            company_data.loc[item["calendarYear"]+item["period"], "Revenue"] = item["revenue"]
            company_data.loc[item["calendarYear"]+item["period"], "OperatingIncome"] = item["operatingIncome"]
            company_data.loc[item["calendarYear"]+item["period"], "EPSDiluted"] = item["epsdiluted"]
            company_data.loc[item["calendarYear"]+item["period"], "GrossProfit"] = item["grossProfit"]
            company_data.loc[item["calendarYear"]+item["period"], "GrossProfitRatio"] = item["grossProfitRatio"]
    if response_is.status_code != 200:
        errors_q.append(i)
    time.sleep(0.02)
    
    url_bs = "https://fmpcloud.io/api/v3/balance-sheet-statement/{}?period=quarter&limit={}&apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i, quarter_count)
    response_bs = requests.get(url_bs)
    for item in response_bs.json():
        if int(item["calendarYear"]) >= year_limit:
            company_data.loc[item["calendarYear"]+item["period"], "Year"] = item["calendarYear"]
            company_data.loc[item["calendarYear"]+item["period"], "Quarter"] = item["period"]
            company_data.loc[item["calendarYear"]+item["period"], "FillingDate"] = item["fillingDate"]
            company_data.loc[item["calendarYear"]+item["period"], "Date"] = item["date"]
            company_data.loc[item["calendarYear"]+item["period"], "TotalAssets"] = item["totalAssets"]
            company_data.loc[item["calendarYear"]+item["period"], "CommonStockEquity"] = item["totalStockholdersEquity"]
            company_data.loc[item["calendarYear"]+item["period"], "CurrentAssets"] = item["totalCurrentAssets"]
            company_data.loc[item["calendarYear"]+item["period"], "CurrentLiability"] = item["totalCurrentLiabilities"]
            company_data.loc[item["calendarYear"]+item["period"], "TotalLiability"] = item["totalLiabilities"]
            company_data.loc[item["calendarYear"]+item["period"], "Inventory"] = item["inventory"]
            company_data.loc[item["calendarYear"]+item["period"], "RetainedEarnings"] = item["retainedEarnings"]
    if response_bs.status_code != 200:
        errors_q.append(i)
    time.sleep(0.02)

    if response_is.json():
        date_map = company_data[["Year", "Quarter", "Date"]].set_index("Date")
    else:
        file = "{}_fundamentals_Quarter.csv".format(i)
        company_data.to_csv(data_path / 'company financials'/i/file)
        print(i, "Done")
        errors_q.append(i+" no data")
        continue

    # url_kr_ttm = "https://fmpcloud.io/api/v3/key-metrics-ttm/{}?limit=40&apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i)
    # response_kr_ttm = requests.get(url_kr_ttm)
    # if response_kr_ttm.json():
    #     for item in ['bookValuePerShareTTM', 'currentRatioTTM','debtToAssetsTTM','debtToEquityTTM', 'roicTTM','shareholdersEquityPerShareTTM','roeTTM','priceToSalesRatioTTM','peRatioTTM','pbRatioTTM','payoutRatioTTM','inventoryTurnoverTTM','enterpriseValueTTM','dividendYieldTTM']:
    #         company_data.loc["TTM", item[0:-3]] = response_kr_ttm.json()[0][item]
    # else:
    #     errors.append(i)
    # if response_kr_ttm.status_code != 200:
    #     errors.append(i)
    # time.sleep(0.06)

    url_kr = "https://fmpcloud.io/api/v3/key-metrics/{}?period=quarter&limit={}&apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i, quarter_count)
    response_kr = requests.get(url_kr)
    for item in response_kr.json():
        year = int(item["date"][0:4])
        if year >= year_limit:
            if item["date"] not in date_map.index:
                if item["date"][5:7] in ["01", "02", "03", "04", "05"] and item["period"] == "Q4" and item not in ["BGI", "WLY"]:
                    date_map.loc[item["date"], "Quarter"] = item["period"]
                    date_map.loc[item["date"], "Year"] = str(int(item["date"][0:4])-1)
                elif item["date"][5:7] in ["08", "09", "10", "11", "12"] and item["period"] == "Q1":
                    date_map.loc[item["date"], "Quarter"] = item["period"]
                    date_map.loc[item["date"], "Year"] = str(int(item["date"][0:4])+1)
                elif item["date"][5:7] in ["08", "09", "10", "11", "12"] and item["period"] == "Q2":
                    date_map.loc[item["date"], "Quarter"] = item["period"]
                    date_map.loc[item["date"], "Year"] = str(int(item["date"][0:4])+1)
                else:
                    date_map.loc[item["date"], "Quarter"] = item["period"]
                    date_map.loc[item["date"], "Year"] = item["date"][0:4]
                errors_q.append(i+" key-metrics " + item["date"] + " " + item["period"])
            index = date_map.loc[item["date"], "Year"]+date_map.loc[item["date"], "Quarter"]
            for key in ["marketCapTTM","enterpriseValueTTM",'bookValuePerShareTTM', 'currentRatioTTM','debtToAssetsTTM','debtToEquityTTM', 'roicTTM','shareholdersEquityPerShareTTM','roeTTM','priceToSalesRatioTTM','peRatioTTM','pbRatioTTM','payoutRatioTTM','inventoryTurnoverTTM','enterpriseValueTTM','dividendYieldTTM']:
                company_data.loc[index, key[0:-3]] = item[key[0:-3]]
    if response_kr.status_code != 200:
        errors_q.append(i)
    time.sleep(0.02)

    url_fr = "https://fmpcloud.io/api/v3/ratios/{}?period=quarter&limit={}&apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i, quarter_count)
    response_fr = requests.get(url_fr)
    for item in response_fr.json():
        year = int(item["date"][0:4])
        if year >= year_limit:
            if item["date"] not in date_map.index:
                date_map.loc[item["date"], "Quarter"] = item["period"]
                date_map.loc[item["date"], "Year"] = item["date"][0:4]
                errors_q.append(i+" ratios " + item["date"]+ " " + item["period"])
            index = date_map.loc[item["date"], "Year"]+date_map.loc[item["date"], "Quarter"]
            for key in ['assetTurnoverTTM',"quickRatioTTM", 'returnOnAssetsTTM','priceToFreeCashFlowsRatioTTM','grossProfitMarginTTM']:
                company_data.loc[index, key[0:-3]] = item[key[0:-3]]
    if response_fr.status_code != 200:
        errors_q.append(i)
    time.sleep(0.02)

    # url_fr_ttm = "https://fmpcloud.io/api/v3/ratios-ttm/{}?apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i)
    # response_fr_ttm = requests.get(url_fr_ttm)
    # if response_fr_ttm.json():
    #     for item in ['assetTurnoverTTM',"quickRatioTTM", 'returnOnAssetsTTM','priceToFreeCashFlowsRatioTTM','grossProfitMarginTTM']:
    #         company_data.loc["TTM", item[0:-3]] = response_fr_ttm.json()[0][item]
    # else:
    #     errors.append(i)
    # if response_fr_ttm.status_code != 200:
    #     errors.append(i)
    # time.sleep(0.06)

    url_fsg = "https://fmpcloud.io/api/v3/financial-growth/{}?period=quarter&limit={}&apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i, quarter_count)
    response_fsg = requests.get(url_fsg)
    for item in response_fsg.json():
        year = int(item["date"][0:4])
        if year >= year_limit:
            if item["date"] not in date_map.index:
                if item["date"][5:7] in ["01", "02", "03", "04", "05"] and item["period"] == "Q4" and item not in ["BGI", "WLY"]:
                    date_map.loc[item["date"], "Quarter"] = item["period"]
                    date_map.loc[item["date"], "Year"] = str(int(item["date"][0:4])-1)
                elif item["date"][5:7] in ["08", "09", "10", "11", "12"] and item["period"] == "Q1":
                    date_map.loc[item["date"], "Quarter"] = item["period"]
                    date_map.loc[item["date"], "Year"] = str(int(item["date"][0:4])+1)
                elif item["date"][5:7] in ["08", "09", "10", "11", "12"] and item["period"] == "Q2":
                    date_map.loc[item["date"], "Quarter"] = item["period"]
                    date_map.loc[item["date"], "Year"] = str(int(item["date"][0:4])+1)
                else:
                    date_map.loc[item["date"], "Quarter"] = item["period"]
                    date_map.loc[item["date"], "Year"] = item["date"][0:4]
                errors_q.append(i+" financial-growth " + item["date"]+ " " + item["period"])
            index = date_map.loc[item["date"], "Year"]+date_map.loc[item["date"], "Quarter"]
            for key in ['epsdilutedGrowth',"fiveYNetIncomeGrowthPerShare", 'fiveYRevenueGrowthPerShare','fiveYShareholdersEquityGrowthPerShare','grossProfitGrowth', "netIncomeGrowth","operatingIncomeGrowth","threeYNetIncomeGrowthPerShare","threeYRevenueGrowthPerShare","threeYShareholdersEquityGrowthPerShare"]:
                company_data.loc[index, key] = item[key]
    if response_fsg.status_code != 200:
        errors_q.append(i)
    time.sleep(0.02)

    url_cp = "https://fmpcloud.io/api/v3/profile/{}?apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i)
    response_cp = requests.get(url_cp)
    if response_cp.json():
        company_data["Industry"] = response_cp.json()[0]["industry"]
        company_data["Sector"] = response_cp.json()[0]["sector"]
    else:
        errors_q.append(i)
    
    # url_ev = "https://fmpcloud.io/api/v3/enterprise-values/{}?period=quarter&limit={}&apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i, quarter_count)
    # response_ev = requests.get(url_ev)
    # for item in response_ev.json():
    #     year = int(item["date"][0:4])
    #     if year >= year_limit:
    #         if item["date"] not in date_map.index:
    #             date_map.loc[item["date"], "Quarter"] = item["period"]
    #             date_map.loc[item["date"], "Year"] = item["date"][0:4]
    #             errors_q.append(i+" enterprise-values " + item["date"]+ " " + item["period"])
    #         index = date_map.loc[item["date"], "Year"]+date_map.loc[item["date"], "Quarter"]
    #         company_data.loc[index, "MarketCap"] = item["marketCapitalization"]
    #         company_data.loc[index, "EnterpriseValue"] = item["enterpriseValue"]
    # if response_ev.status_code != 200:
    #     errors_q.append(i)
    # time.sleep(0.02)
    
    company_data["Symbol"] = i
    if response_cp.status_code != 200:
        errors_q.append(i)
    time.sleep(0.02)

    company_data = company_data.rename(columns={"enterpriseValue":"EnterpriseValue", "marketCap": "MarketCap"})
    file = "{}_fundamentals_Quarter.csv".format(i)
    company_data.to_csv(data_path /"fundamental data"/ 'company financials'/i/file)
    print(i, "Done")

In [3]:
for year in ["TTM"]:
    year_data = pd.DataFrame()
    year_str = str(year)
    new_path = data_path/"fundamental data"/"year financials"/year_str
    if not new_path.exists():
        new_path.mkdir(parents=True)
    for symbol in [name for name in os.listdir(data_path /"fundamental data"/ 'company financials')]:
        path = data_path /"fundamental data"/ 'company financials'/ symbol
        file = symbol+"_fundamentals_Year.csv"
        if file in [name for name in os.listdir(path)]:
            file_path = data_path /"fundamental data"/ 'company financials' / symbol/file
            comp_data = pd.read_csv(file_path, index_col="Year")
            if year_str in comp_data.index:
                year_data = pd.concat([year_data, comp_data.loc[[year_str]].reset_index()], ignore_index=True)
    year_data = year_data.set_index("Symbol")
    year_file = year_str+"_fundamentals_Year.csv"
    year_data.to_csv(new_path/year_file)

In [4]:
for year in range(2008,2024):
    year_data = pd.DataFrame()
    year_str = str(year)
    new_path = data_path/"fundamental data"/"year financials"/year_str
    if not new_path.exists():
        new_path.mkdir(parents=True)
    for symbol in [name for name in os.listdir(data_path /"fundamental data"/ 'company financials')]:
        path = data_path /"fundamental data"/ 'company financials'/ symbol
        file = symbol+"_fundamentals_Year.csv"
        if file in [name for name in os.listdir(path)]:
            file_path = data_path /"fundamental data"/ 'company financials' / symbol/file
            comp_data = pd.read_csv(file_path, index_col="Year")
            if year_str in comp_data.index:
                year_data = pd.concat([year_data, comp_data.loc[[year_str]].reset_index()], ignore_index=True)
    year_data = year_data.set_index("Symbol")
    year_file = year_str+"_fundamentals_Year.csv"
    year_data.to_csv(new_path/year_file)

In [5]:
for year in range(2008,2024):
    year_str = str(year)
    new_path = data_path/"fundamental data"/"year financials"/year_str
    if not new_path.exists():
        new_path.mkdir(parents=True)
    for q in ["Q1", "Q2","Q3","Q4"]:
        q_data = pd.DataFrame()
        for symbol in [name for name in os.listdir(data_path /"fundamental data"/ 'company financials')]:
            path = data_path/"fundamental data" / 'company financials'/ symbol
            file = symbol+"_fundamentals_Quarter.csv"
            if file in [name for name in os.listdir(path)]:
                file_path = data_path /"fundamental data"/ 'company financials' / symbol/file
                comp_data = pd.read_csv(file_path, index_col=["Year", "Quarter"])
                if (year,q) in comp_data.index:
                    q_data = pd.concat([q_data, comp_data.loc[[(year,q)]].reset_index()], ignore_index=True)
        if not q_data.empty:
            q_data = q_data.set_index("Symbol")
            q_file = year_str+q+"_fundamentals_Q.csv"
            q_data.to_csv(new_path/q_file)

C:\Users\fuway\AppData\Local\Temp\ipykernel_6092\565708974.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  if (year,q) in comp_data.index:
C:\Users\fuway\AppData\Local\Temp\ipykernel_6092\565708974.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  if (year,q) in comp_data.index:
C:\Users\fuway\AppData\Local\Temp\ipykernel_6092\565708974.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  if (year,q) in comp_data.index:
C:\Users\fuway\AppData\Local\Temp\ipykernel_6092\565708974.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  if (year,q) in comp_data.index:
C:\Users\fuway\AppData\Local\Temp\ipykernel_6092\565708974.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  if (year,q) in comp_data.index:
C:\Users\fuway\AppData\Local\Temp\ipykernel_6092\565708974.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  

In [11]:
path_2009 = data_path/"fundamental data"/"year financials"/"2009"/"2009_fundamentals_year.csv"
path_2022 = data_path/"fundamental data"/"year financials"/"2022"/"2022_fundamentals_year.csv"
df_2009 = pd.read_csv(path_2009, index_col="Symbol")
df_2022 = pd.read_csv(path_2022, index_col="Symbol")
[i for i in df_2009.index if i in df_2022.index]

['A',
 'AACG',
 'AAIC',
 'AAL',
 'AAME',
 'AAN',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AAU',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABCM',
 'ABEO',
 'ABEV',
 'ABG',
 'ABIO',
 'ABM',
 'ABR',
 'ABST',
 'ABT',
 'ABUS',
 'ABVC',
 'ACAD',
 'ACB',
 'ACCO',
 'ACER',
 'ACET',
 'ACGL',
 'ACGLN',
 'ACGLO',
 'ACHV',
 'ACIW',
 'ACLS',
 'ACM',
 'ACN',
 'ACNB',
 'ACNT',
 'ACOR',
 'ACR',
 'ACRX',
 'ACTG',
 'ACU',
 'ADBE',
 'ADC',
 'ADEA',
 'ADES',
 'ADI',
 'ADM',
 'ADMA',
 'ADMP',
 'ADP',
 'ADSK',
 'ADTN',
 'ADUS',
 'ADX',
 'ADXN',
 'AE',
 'AEE',
 'AEF',
 'AEG',
 'AEHL',
 'AEHR',
 'AEIS',
 'AEL',
 'AEM',
 'AEMD',
 'AENZ',
 'AEO',
 'AEP',
 'AEPPZ',
 'AER',
 'AES',
 'AESC',
 'AEY',
 'AEZS',
 'AFG',
 'AFGE',
 'AFL',
 'AG',
 'AGCO',
 'AGEN',
 'AGI',
 'AGM',
 'AGNC',
 'AGNCL',
 'AGNCM',
 'AGNCN',
 'AGNCO',
 'AGNCP',
 'AGO',
 'AGR',
 'AGRO',
 'AGX',
 'AGYS',
 'AHT',
 'AIG',
 'AIM',
 'AIMD',
 'AIMDW',
 'AIN',
 'AIR',
 'AIRC',
 'AIRT',
 'AIT',
 'AIV',
 'AIZ',
 'AJG',
 'AJRD',
 'AKAM',
 'AKR',
 'ALB',
 'ALC

In [13]:
path = data_path/"fundamental data"/"year financials"
symbols=[]
for year in range(2009,2022):
    year_str = str(year)
    file_str = year_str+"_fundamentals_year.csv"
    new_path = data_path/"fundamental data"/"year financials"/year_str/file_str
    df = pd.read_csv(new_path, index_col="Symbol").dropna(how="all")
    if not symbols:
        symbols = [i for i in df.index.unique()]
    else:
        symbols = [i for i in df.index.unique() if i in symbols]

In [15]:
year_str = "TTM"
file_str = year_str+"_fundamentals_year.csv"
new_path = data_path/"fundamental data"/"year financials"/year_str/file_str
df = pd.read_csv(new_path, index_col="Symbol").dropna(axis=1, how="all")
symbols = [i for i in symbols if i in df.index.unique()]

In [16]:
len(symbols)

3061

In [18]:
finer_symbols = []
for i in symbols:
    url_cp = "https://fmpcloud.io/api/v3/profile/{}?apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i)
    response_cp = requests.get(url_cp)
    if response_cp.json()[0]["range"]:
        if float(response_cp.json()[0]["range"].split("-")[0]) > 3:
            finer_symbols.append(i)

In [19]:
len(finer_symbols)

2489

In [20]:
data_2022 = pd.read_csv(data_path/"fundamental data"/"year financials"/"2022"/"2022_fundamentals_year.csv", index_col="Symbol").dropna(axis=1, how="all")
finer_stock = data_2022[data_2022.index.isin(finer_symbols)][["Sector", "Industry"]]
finer_stock.to_csv(data_path/"fundamental data"/"Finer_Stock.csv")

In [113]:
from datetime import date
today = date.today()
path = data_path/"stock price data"/"company stock price daily"
for i in [name for name in os.listdir(data_path /"fundamental data"/ 'company financials')]:
    file_str=i+".csv"
    comp_stock = pd.DataFrame()
    url_sp = "https://fmpcloud.io/api/v3/historical-price-full/{}?from=2010-01-01&to={}&apikey=2e59b0f3eae2bfdf48ab4c673b5c3aee".format(i, today.strftime("%Y-%m-%d"))
    response_sp = requests.get(url_sp)
    comp_stock = pd.DataFrame(response_sp.json()["historical"])
    comp_stock["Symbol"] = i
    comp_stock.to_csv(path/file_str, index=False)
    

In [23]:
stock_symbol = []
path = data_path/"stock price data"/"company stock price daily"
for i in [name for name in os.listdir(data_path /"fundamental data"/ 'company financials')]:
    file_str=i+".csv"
    comp_stock = pd.read_csv(path/file_str)
    if len(comp_stock) == 3402:
        stock_symbol.append(i)
data_complete_stock = data_2022[data_2022.index.isin(stock_symbol)][["Sector", "Industry"]]
data_complete_stock.to_csv(data_path/"stock price data"/"data_complete_stock.csv")

In [115]:
stock_symbol = []
for i in finer_symbols:
    file_str=i+".csv"
    comp_stock = pd.read_csv(path/file_str)
    if len(comp_stock) == 3402:
        stock_symbol.append(i)
price_stock = finer_stock[finer_stock.index.isin(stock_symbol)]
price_stock.to_csv(data_path/"stock price data"/"finer_data_complete_stock.csv")